https://earthquake.usgs.gov/earthquakes/search/

In [1]:
from datetime import datetime

from parser.app import CsvParser

In [2]:
start = datetime(2022,3,6)
end = datetime(2023,6,15)


parser = CsvParser(start, end, 'world', 'World_2000-2023')
parser.to_csv()

start (datetime.datetime(2022, 3, 6, 0, 0), datetime.datetime(2022, 12, 31, 0, 0)) period
2022-03-06 00:00:00
start (datetime.datetime(2023, 1, 1, 0, 0), datetime.datetime(2023, 6, 15, 0, 0)) period
2023-01-01 00:00:00


True